# Importing Data and defining functions for feature engineering

1. Import Modules
2. Import data
3. Defining of functions
4. Applying and testing

## Importing Modules

In [3]:
import pandas as pd
import numpy as np

import os
import sys
import logging

parentdir = os.path.dirname(os.path.abspath(''))
sys.path.append(parentdir)

import functions as fu
from feature_engineering import *

# Pfade setzen
c_fold = fu.get_folder()
p_fold = c_fold + '\data\Prepared'
folder = c_fold + '\data\Processed'


# Initialize Logging
logging.basicConfig(format='[%(asctime)s] %(levelname)s: %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.INFO,
                    handlers=[
                        logging.FileHandler(
                            filename='datalogger.log',
                            mode='a',
                            encoding='utf-8'),
                        logging.StreamHandler(sys.stdout)])

# Importing Data and calculating the features
* für jeden zyklus die anzahl der vorherigen entlade und ladezyklen anzeigen
* filtern auf die entladezyklen da dies das trainingmaterial ist.

In [4]:
# import Full Data
#df = fu.csv_einlesen(p_fold)


# import one dataset for developing
for file in os.listdir(p_fold):
    if ".csv" in file:
        logging.info(f"<<< Started processing file: {file} >>>")
        df = pd.read_csv(f"{p_fold}\\{file}", index_col = 0,
                         #nrows= 100000
                        )
        logging.info(f"<<< Size of Dataframe: {df.shape} >>>")
        # create efficient  datetypes for more efficient memory usage
        df[["batteryname",'comment','type']] = df[["batteryname",'comment','type']].astype("category")
        df[['time','relativeTime','voltage','current','temperature']] = df[['time','relativeTime','voltage','current','temperature']].apply(pd.to_numeric, downcast="float")
        df[['zyklus','step']] = df[['zyklus','step']].apply(pd.to_numeric, downcast="integer")
        
        # applying new features
        df = feature_creator(df,['voltage','current','temperature'],['batteryname','zyklus'],"mean")
        df = feature_creator(df,['voltage','current','temperature'],['batteryname','zyklus'],"median")
        df = feature_creator(df,['voltage','current','temperature'],['batteryname','zyklus'],"min")
        df = feature_creator(df,['voltage','current','temperature'],['batteryname','zyklus'],"max")
        df = feature_creator(df,['voltage','current','temperature'],['batteryname','zyklus'],"std")
        
        # calculate numbers of discharging and charging zycles
        df['type_d'] = calc_previous_cycles(df,columns = ['batteryname','zyklus'], types = 'D')
        df['type_c'] = calc_previous_cycles(df,columns = ['batteryname','zyklus'], types = 'C')
        # applying calculated remaining time in zyklus
        df['target'] = target_creator(df)
        
        df = df[df.type == 'D']
        logging.info(f"<<< Start saving dataframe with shape {df.shape} >>>")
        df.to_csv(f"{folder}\\{file}", sep=',')
        logging.info(f"<<< Saved processed file in directory {folder}\\{file} >>>")

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

data = df

corrMatrix = data.corr()
mask = np.triu(np.ones_like(corrMatrix, dtype=bool))


plt.figure(figsize=(15,10))
sns.heatmap(corrMatrix, annot=True,mask=mask)
plt.draw()

NameError: name 'df' is not defined